In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
import os

src_path = os.getcwd()

In [2]:
stop_words = [word.strip() for word in open(src_path+'/stop_words.txt').readlines()]

with open(src_path+'/dem.txt', 'r') as file:
    dem_text = [line.strip('\n') for line in file]
    
with open(src_path+'/gop.txt', 'r') as file:
    gop_text = [line.strip('\n') for line in file]

In [12]:
#문장의 매트릭스화를 진행 (첫 800단어만)
vectorizer = CountVectorizer(input=dem_text+gop_text,
                            stop_words=stop_words,
                            max_features=800)
dem_bow = vectorizer.fit_transform(dem_text)
gop_bow = vectorizer.fit_transform(gop_text)

In [13]:
(dem_bow.shape, gop_bow.shape)

((200, 800), (200, 800))

In [14]:
#모델의 훈련을 위한 배열 제공
x = sparse.vstack((dem_bow, gop_bow))

#레이블 벡터를 만들기 위해 dem은 1, gop는 0인 벡터를 조립
ones = np.ones(200)
zeros = np.zeros(200)
y = np.hstack((ones, zeros))

In [15]:
x.shape, y.shape

((400, 800), (400,))

In [16]:
#모델의 훈련 전, 트윗들을 무작위로 분할하여 75%는 모델을 만드는데 사용하고, 나머지는 예측이 잘 되는지를 확인하는데 사용
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [17]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((300, 800), (100, 800), (300,), (100,))

In [18]:
#훈련 및 테스트셋의 준비가 되었기 때문에 naive bayes(0과 1이므로 bernoulli naive bayes)로 모델 훈련 진행
from sklearn.naive_bayes import BernoulliNB
naive_bayes = BernoulliNB()
model = naive_bayes.fit(x_train, y_train)

In [21]:
#예측 값을 얻음
y_predictions = model.predict(x_test)

In [22]:
y_predictions, y_test

(array([0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
        0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.]),
 array([0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
        0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.]))

In [24]:
#예측 값과 얼마나 일치하는지 확인 (정확도))
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predictions)

1.0

In [26]:
#파이프라인을 이용한 모델링
from sklearn.pipeline import Pipeline

x_train, x_test, y_train, y_test = train_test_split(dem_text+gop_text,y,
                                                   test_size=0.25,
                                                   random_state=5)
pipeline = Pipeline([('vect', vectorizer), ('nb', naive_bayes)])
pipeline_model = pipeline.fit(x_train, y_train)
y_predictions = pipeline_model.predict(x_test)
accuracy_score(y_test, y_predictions)

0.93